> <picture>
>   <source media="(prefers-color-scheme: light)" srcset="https://raw.githubusercontent.com/Mqxx/GitHub-Markdown/main/blockquotes/badge/light-theme/issue.svg">
>   <img alt="Issue" src="https://raw.githubusercontent.com/Mqxx/GitHub-Markdown/main/blockquotes/badge/dark-theme/issue.svg">
> </picture><br>
>
> **Issue**
> 
>  Implement solving of Queens Puzzle with evolutionary algorithm. For example, [genetic algorithm](https://en.wikipedia.org/wiki/Eight_queens_puzzle).
> 
> The goal is to design algorithm that would solve queens puzzle with at least 8x8 board. But you can try bigger sizes. As a pattern you can use the same script as for 1st assignment, but modify all required things to work with discrete problem. It is important to design how you will encode solution (how you represent a solution in a code)  and design correspond mutation\crossover operators. 
Try to print in console your best found solution at the end of your script.

In [29]:
!pip install deap


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [30]:
from itertools import cycle

import math
import random

from multiprocessing import Pool

import pandas as pd
import numpy as np
import numpy.random as rnd

import deap
from deap import tools, base, creator, algorithms

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [31]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

/opt/python/envs/default/lib/python3.8/site-packages/deap/creator.py:185: RuntimeWarning:

A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/opt/python/envs/default/lib/python3.8/site-packages/deap/creator.py:185: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [32]:
class QueensExperiment:
    def __init__(self, queens, fitness_evaluation, population_size, generations, crossover_rate, mutation_rate, mutation_function):
        self.individual_generator = lambda: random.choices(range(queens**2), k=queens)
        self.fitness_evaluation = fitness_evaluation
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate  = mutation_rate
        self.mutation_function  = mutation_function

        self.toolbox = base.Toolbox()
        self.toolbox.register("individual", tools.initIterate, creator.Individual, self.individual_generator)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)

        # fitness evaluation function
        self.toolbox.register("evaluate", lambda individual: (fitness_evaluation(individual),))

        # mutators 
        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", mutation_function[0], **mutation_function[1])

        # selection function
        self.toolbox.register("select", tools.selTournament, tournsize=4)


    def run(self):
        pop = self.population_size
        # Fittest individual with a hall of fame record
        hof = tools.HallOfFame(3, np.array_equal)
        
        # generation statistics logging
        stats = tools.Statistics(lambda ind: ind.fitness.values[0])
        
        stats = tools.Statistics(key=lambda individual: individual.fitness.values)
        stats.register("std", lambda population: np.std([fitness for fitness in population if fitness[0] != math.inf]))
        stats.register("avg", lambda population: np.mean([fitness for fitness in population if fitness[0] != math.inf]))
        stats.register("min", lambda population: np.min([fitness for fitness in population if fitness[0] != math.inf]))
        stats.register("max", lambda population: np.max([fitness for fitness in population if fitness[0] != math.inf]))

        

        _, log = algorithms.eaSimple(
            self.toolbox.population(n=pop),
            self.toolbox,
            ngen=generations,
            cxpb=crossover_rate, 
            mutpb=mutation_rate,
            stats=stats, 
            halloffame=hof, 
            verbose=False,
        )

        print("Best = {}".format(hof[0]))
        print("Best fit = {}".format(hof[0].fitness.values[0]))
        return log, hof[0]

In [33]:
def starigin(individual):
    #  removed duplicate values 
    if len(individual) != len(set(individual)):
        return math.inf

    dimension = len(individual)
    # count all pairs of conflicts
    fitness: float = 0
    for x in range(len(individual)):
        x_row, x_column = individual[x] // dimension, individual[x] % dimension
        for y in range(x + 1, len(individual)):
            y_row, y_column = individual[y] // dimension, individual[y] % dimension
            if x_row == y_row or x_column == y_column or abs(x_row - y_row) == abs(x_column - y_column):
                fitness += 1
    
    return fitness

In [34]:
QUEENS_NUMBER = 8    
generations = 100 # set it to 100
pop_size = 2500      # lets say max is 100
crossover_rate=0.5
mutation_rate=0.5
mutation_function= (
    tools.mutUniformInt, 
    {"low": 0, "up": 8**2 - 1, "indpb": 1/4}
)

scenario = QueensExperiment(QUEENS_NUMBER, starigin, pop_size, generations, crossover_rate, mutation_rate, mutation_function)
log, fittest = scenario.run()
# draw_log(log)

Best = [26, 57, 46, 52, 13, 32, 3, 23]
Best fit = 0.0


In [36]:
def plot_curves(generation, deviation, average, minimum, maximum):
    fig = go.Figure()
    palette = cycle(px.colors.qualitative.Plotly)

    # Add traces
    fig.add_trace(go.Scatter(
        x=generation, y=deviation,
        name="Deviation",
        marker_color=next(palette),
    ))

    fig.add_trace(go.Scatter(
        x=generation, y=average,
        name="Average",
        marker_color=next(palette),
    ))

    fig.add_trace(go.Scatter(
        x=generation, y=minimum,
        name="Min",
        marker_color=next(palette),
    ))

    fig.add_trace(go.Scatter(
        x=generation, y=maximum,
        name="Max",
        marker_color=next(palette),
    ))

    fig.update_layout(
            title="Solving of Queens Puzzle",
            xaxis_title="Generation",
            yaxis_title="Fitness",
            legend_title="Who 🗿?",
    )
    fig.show()


plot_curves(*log.select("gen", "std", "avg", "min", "max"))

In [49]:
print(f"Pos: {fittest}")
print(f"Dups: {len(fittest) - len(set(fittest))}")

Pos: [26, 57, 46, 52, 13, 32, 3, 23]
Dups: 0


In [47]:
def display_grid(individual):
    dimension = len(individual)
    board = pd.DataFrame("", index=range(1, dimension+1), columns=range(1, dimension+1))

    for x in range(dimension):
        x_row, x_col = individual[x] // dimension, individual[x] % dimension
        for y in range(x+1, dimension):
            y_row, y_col = individual[y] // dimension, individual[y] % dimension
            if (x_row == y_row and x_col == y_col) or (abs(x_row-y_row) == abs(x_col-y_col)):
                board[1+x_col][1+x_row] = "❌"

    for queen in individual:
        row, col = queen // dimension, queen % dimension
        board[1+col][1+row] = "🗿" if board[1+col][1+row] == "" else "🗿"

    print(board)

> <picture>
>   <source media="(prefers-color-scheme: light)" srcset="https://raw.githubusercontent.com/Mqxx/GitHub-Markdown/main/blockquotes/badge/light-theme/complete.svg">
>   <img alt="Complete" src="https://raw.githubusercontent.com/Mqxx/GitHub-Markdown/main/blockquotes/badge/dark-theme/complete.svg">
> </picture><br>
>
> Solved Queens Puzzle

In [48]:
display_grid(fittest)

   1  2  3  4  5  6  7  8
1           🗿            
2                 🗿      
3                       🗿
4        🗿               
5  🗿                     
6                    🗿   
7              🗿         
8     🗿                  
